In [0]:
def read_dataset(year_range = [2015, 2016, 2017], month_range = range(1, 13), comments_or_submissions = "comments"):
    if comments_or_submissions == "comments":
        link = "comments_tree_shaking/"
    elif comments_or_submissions == "submissions":
        link = "kwfilter_top3/"
    else:
        link = "non-exist"
    result = None
    for year in year_range:
        for month in month_range:
            month_literal = str(month)
            data_path = "/mnt/lsde/group05/" + link + str(year) + "-" + month_literal + ".parquet"
            if result is None:
                result = spark.read.parquet(data_path)
            else:
                result = result.union(spark.read.parquet(data_path))
    return result

In [0]:
from pyspark.sql.functions import col, concat, lit

def innerjoin(s_year = 2014, s_month = 2):
    submissions = read_dataset([s_year], [s_month], "submissions")
    if (s_month != 12):
        if (s_month == 8 and s_year == 2022):
            comments = read_dataset([s_year], [s_month], "comments")
        else:
            comments = read_dataset([s_year], [s_month, s_month + 1], "comments")
    else:
        comments_1 = read_dataset([s_year], [s_month], "comments")
        comments_2 = read_dataset([s_year + 1], [1], "comments")
        comments = comments_1.union(comments_2)
    submissions_id = submissions.select("id", "title", "selftext")
    # print(comments.count())
    submissions_id = submissions_id.withColumn("id", concat(lit("t3_"), col("id"))).withColumnRenamed("id", "parent_id")
    on_topic_comments = submissions_id.join(comments, "parent_id", "inner")
    # print(on_topic_comments.count())
    on_topic_comments.write.mode("overwrite").format("parquet").save("/mnt/lsde/group05/filtered_comments_2/" + str(s_year) + "-" + str(s_month) + ".parquet")

In [0]:
for year in range(2015, 2022):
    month = 12
    print("join " + str(year) + "-" + str(month))
    innerjoin(s_year = year, s_month = month)

join 2015-12
join 2016-12
join 2017-12
join 2018-12
join 2019-12
join 2020-12
join 2021-12

In [0]:
for year in range(2019, 2023):
    for month in range(1, 12):
        print("join " + str(year) + "-" + str(month))
        innerjoin(s_year = year, s_month = month)

In [0]:
def output():
    for month in range(2, 13):
        print("join 2014-" + str(month))
        innerjoin(s_year = 2014, s_month = month)
    for year in range(2015, 2022):
        for month in range(1, 13):
            print("join " + str(year) + "-" + str(month))
            if (month != 12):
                innerjoin(s_year = year, s_month = month)
    for month in range(1, 9):
        print("join 2022-" + str(month))
        innerjoin(s_year = 2022, s_month = month)

In [0]:
output()

In [0]:
# from pyspark.sql.functions import col, concat, lit

# submissions = spark.read.parquet("/mnt/lsde/group05/kwfilter_top3/*.parquet")
# comments = spark.read.parquet("/mnt/lsde/group05/comments_tree_shaking/*.parquet")
# submissions_id = submissions.select("id")
# submissions_id = submissions_id.withColumn("id", concat(lit("t3_"), col("id"))).withColumnRenamed("id", "parent_id")
# on_topic_comments = submissions_id.join(comments, "parent_id", "inner")
# on_topic_comments.write.mode("overwrite").format("parquet").save("/mnt/lsde/group05/filtered_comments/comments.parquet")